In [1]:
#!pip install requests

#!pip install beautifulsoup4

#!pip install lxml

#!pip install selenium

In [3]:
import requests
from bs4 import BeautifulSoup

import urllib
from urllib.request import urlretrieve

import json

import os

In [4]:
#codi 모음 페이지 (1491부터 시작), collection number는 코디북 사이트에서 제공 https://ko.codibook.net/collection?mode=list
def crawl_codibook_collection(collection_num_begin, collection_num_end):
    url_codi_collection = 'https://ko.codibook.net/collection/'
    for i in range(collection_num_begin, collection_num_end+1):
        url = url_codi_collection + f'{i}'
        headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36'}
        res = requests.get(url, headers = headers)
        soup = BeautifulSoup(res.text, 'lxml')
        null_msg = soup.find(class_='error_message')
        
        #없는 page면 break
        if null_msg is not None:
            continue
        

        #코디 속 아이템이 판매중인 코디 검색
        codi_items = soup.find_all('div', class_ = "codi_list_items_wrapper")

        for codi_item_wrapper in codi_items:
            #코디 속 아이템을 판매하는 코디가 없는 경우 break
            if codi_item_wrapper.find('a') is None:
                continue
            codi = codi_item_wrapper.parent.parent.find(class_='codi_link')
            codi_link  = 'https://ko.codibook.net' + codi['href']
            codi_id = codi['data-codi_id']
            codi_img = codi.find(class_ = 'thumb')['src']
            
            #codi의 info, img 저장
            urllib.request.urlretrieve(codi_img,f'./codibook/img/img_style/codibook_style_{codi_id}.jpg' )
            
            codi_info ={}
            codi_info['style_id'] = codi_id
            codi_info['link'] = codi_link    
            with open(f'./codibook/info/info_style/codibook_style_{codi_id}.json', 'w', encoding='utf-8') as make_file:
                json.dump(codi_info, make_file, indent="\t", ensure_ascii=False)
            make_file.close()
            
            
            #codi속 item 추출
            codi_item_list = codi_item_wrapper.find_all('a')
            for codi_item in codi_item_list:
                item_info = {}
                if codi_item.find(class_= 'codi_item_title') is not None:
                    item_info['title'] = codi_item.find(class_= 'codi_item_title').get_text()
                
                item_info['link'] = 'https://ko.codibook.net' + codi_item['href']
                item_id = codi_item['href'].strip('/item/')
                item_info['item_id'] = item_id
                item_info['style_id'] = codi_id
                if codi_item.find('img') is None:
                    continue
                item_img = codi_item.find('img')['src']
                
                #item의 img, info 저장
                urllib.request.urlretrieve(item_img,f'./codibook/img/img_item/codibook_item_{codi_id}_{item_id}.jpg' )
                
                with open(f'./codibook/info/info_item/codibook_item_{codi_id}_{item_id}.json', 'w', encoding='utf-8') as make_file:
                    json.dump(item_info, make_file, indent="\t", ensure_ascii=False)
                make_file.close()

In [ ]:
crawl_codibook_collection(2000,2340)